# Задание 1
Используем файл keywords.csv.
 
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
 
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
 
geo_data = {
 
    'Центр': ['москва', 'тула', 'ярославль'],
 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
}
 
Результат классификации запишите в отдельный столбец region.

In [2]:
import pandas as pd
keywords=pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [3]:
geo_data = {
 
    'Центр': ['москва', 'тула', 'ярославль'],
 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
}

In [4]:
def  keyword_region(keyword):
    for key, items in geo_data.items():
        for i in keyword.split():
            if i in items:
                return key
    return 'undefined'
           
keywords['region'] = keywords['keyword'].apply(lambda row: keyword_region(row))

In [5]:
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [6]:
keywords[(keywords['region']!='undefined')].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


# Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
 
Результат классификации запишите в столбец class

In [7]:
import pandas as pd
ratings=pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
ratings = ratings.groupby('movieId').mean()
ratings.head()

,userId,rating,timestamp
movieId,,,
1,338.558704,3.872470,1.103116e+09
2,318.906542,3.401869,1.069321e+09
3,374.423729,3.161017,9.662429e+08
4,355.538462,2.384615,9.277797e+08
5,320.785714,3.267857,9.967201e+08


In [9]:
def film_classification(row):
    if row['rating']<=2:
        return 'Низкий рейтинг'
    elif row['rating']<=4 and row['rating']>2:
        return 'Средний рейтинг'
    elif row['rating']>4.5:
        return 'Высокий рейтинг'

In [10]:
ratings['class']=ratings.apply(film_classification, axis=1)

Соединим с таблицей, содержащей информацию о фильме. Выведем только столбцы movieid, title, rating и class

In [11]:
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
movies_and_raitings_class = movies.merge(ratings, left_on='movieId', right_on='movieId', how='inner')[['movieId','title','rating','class']]

In [13]:
movies_and_raitings_class.head()

,movieId,title,rating,class
0,1,Toy Story (1995),3.872470,Средний рейтинг
1,2,Jumanji (1995),3.401869,Средний рейтинг
2,3,Grumpier Old Men (1995),3.161017,Средний рейтинг
3,4,Waiting to Exhale (1995),2.384615,Средний рейтинг
4,5,Father of the Bride Part II (1995),3.267857,Средний рейтинг


# Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [14]:
ratings=pd.read_csv('ratings.csv')
new_ratings=ratings.groupby( 'userId').count().reset_index()[['userId','movieId']]
new_ratings=new_ratings[(new_ratings['movieId']>=100)]
new_ratings.head()

,userId,movieId
3,4,204
4,5,100
7,8,116
14,15,1700
16,17,363


In [15]:
film_fans_user_ids = new_ratings['userId'].tolist()

In [16]:
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
fans_data['timestamp'] = pd.to_datetime(fans_data['timestamp'],unit='s')
fans_data.head()

/Users/lera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,movieId,rating,timestamp
147,4,10,4.0,2000-02-06 04:17:25
148,4,34,5.0,2000-02-07 10:32:36
149,4,112,5.0,2000-02-06 04:16:22
150,4,141,5.0,2000-02-07 10:34:41
151,4,153,4.0,2000-02-06 04:29:06


Избавимся от столбцов 'rating' и 'movieId', так как для дальнейшей работы нам нужен только столбец 'timestamp'

In [17]:
fans_data.drop(['rating','movieId'],axis=1, inplace=True)

/Users/lera/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
fans_data.head()

,userId,timestamp
147,4,2000-02-06 04:17:25
148,4,2000-02-07 10:32:36
149,4,2000-02-06 04:16:22
150,4,2000-02-07 10:34:41
151,4,2000-02-06 04:29:06


Теперь выведем минимальное и максимальное значения 'timestamp' для каждого пользователя

In [19]:
user_time=fans_data.groupby('userId').agg({'timestamp': ['min', 'max']}).reset_index()

In [20]:
user_time.head()

userId           timestamp                    
                         min                 max
0      4 2000-02-05 19:25:14 2000-02-08 03:57:54
1      5 2006-11-12 23:10:44 2006-11-12 23:45:45
2      8 2006-07-31 23:42:20 2006-08-01 23:22:07
3     15 2001-08-16 04:47:19 2016-07-24 03:25:35
4     17 2005-09-23 09:43:07 2005-09-23 11:57:20

Создадим столбец 'lifetime', который будет содержать время жизни каждого пользователя

In [21]:
def lifetime(row):
    return (row['timestamp']['max']-row['timestamp']['min'])

In [22]:
user_time['lifetime']=user_time.apply(lifetime, axis=1)

In [23]:
user_time.head()

userId           timestamp                               lifetime
                         min                 max                   
0      4 2000-02-05 19:25:14 2000-02-08 03:57:54    2 days 08:32:40
1      5 2006-11-12 23:10:44 2006-11-12 23:45:45    0 days 00:35:01
2      8 2006-07-31 23:42:20 2006-08-01 23:22:07    0 days 23:39:47
3     15 2001-08-16 04:47:19 2016-07-24 03:25:35 5455 days 22:38:16
4     17 2005-09-23 09:43:07 2005-09-23 11:57:20    0 days 02:14:13

А теперь находим среднее значение Lifetime для киноманов 

In [24]:
user_time['lifetime'].mean()

Timedelta('455 days 05:29:08.566539')

# Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
1. В переменную years запишите список из всех годов с 1950 по 2010.
 
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [25]:
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
ratings=pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [27]:
ratings_and_movies = ratings.merge(movies, on='movieId', how='left')
ratings_and_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [28]:
len(ratings) == len(ratings_and_movies)

True

In [29]:
years=[i for i in range(1950,2011)]

In [30]:
def production_year(title):
    for i in years:
        if str(i) in title:
            production_year=i
            break
        else:
            production_year=1900
    return production_year

In [31]:
ratings_and_movies['year']=ratings_and_movies['title'].apply(lambda x: production_year(x))

In [32]:
ratings_and_movies.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


Выведем средний рейтинг фильмов по годам

In [33]:
ratings_and_movies.groupby('year').agg({'rating': 'mean'}).reset_index().sort_values('rating', ascending=False).head(15)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
25,1974,3.978704
13,1962,3.952446
1,1950,3.915254
28,1977,3.905786
15,1964,3.841492
